# Physical Space and the Image Grid

# 物理空间与图像网格

### Learning Objectives

* Understand how an **image pixel grid** is defined in **physical space**
* Understand how a **spatial transform** *defines the map between two physical spaces*
* Understand **how to define a resampling operation** to generate a new image grid
* Understand the **role of image interpolators**, and become familiar with a few common interpolators

### 学习目标

* 理解**图像像素网格**如何定义在**物理空间**中
* 理解**空间变换**定义了*两个物理空间的变换*
* 理解**如何定义一个重采样操作**来生成新的图像网格
* 理解**图像插值的角色**，熟悉通用的插值方法

## Overview

### Physical Space, Images, and the Pixel Grid

### 物理空间，图像，和像素网格

When performing image registration, where the object is to find the spatial transform that maps between the fixed image space and the moving image space, it is **extremely important** to have clear definitions of **physical space**, aka **world space**, and how images live in these spaces.

Once we have *good definitions* and *follow them consistently*, operations such resampling, interpolation, multi-resolution processing, and spatial transform as are **clear** and **correct**, even on images that **anisotropic**, **oriented**, or with **multi-component tensor pixels**, such as gradient vectors or diffusion tensors.

处理图像配准的时候，当对象要找到固定图像空间和移动图像空间之间的空间变换映射时，能够清楚的定义图像如何在对应的物理空间（或世界空间）中是非常重要的。

一旦我们有了好的定义并始终如一地遵循它们，即使在各向异性的、定向的或具有多分量张量像素（例如梯度向量或扩散张量）的图像上，诸如重新采样、插值、多分辨率处理和空间变换等操作也是清晰和正确的。

### Problem: Register a Human Lung Volume from Two Time Points

Given a problem to register x-ray computed tomography (CT) images of a human lung taken at two time points, **what must be supported?**

In image registration, we typically identify the two images as the **fixed** and **moving** image. Our goal is to **find the spatial transformation that makes the moving image so align with the fixed image**. Our fixed image was acquired from the patient at a baseline time point, and our moving image was acquired at a follow-up time point.

### 问题：来自两个时间点的人类肺图像配准

考虑到在两个时间点拍摄的人肺x射线计算机断层扫描（CT）图像的配准问题，**必须支持什么**？

在图像配准的时候，我们通常两个图像分别定义为**固定**图像和**运动**图像。我们的目标是**找到使运动图像与固定图像对齐的空间变换**。固定图像从病人的baseline时间点获取，我们的运动图像从随后的时间点获取。

First, let's examine our fixed and moving images.

首先，让我们来看一下固定和运动图像

In [1]:
import itk
from itkwidgets import view, checkerboard

In [2]:
fixed_image = itk.imread('data/CT_3D_lung_fixed.mha')

view(fixed_image, mode='x')

Viewer(geometries=[], gradient_opacity=0.22, mode='x', point_sets=[], rendered_image=<itkImagePython.itkImageS…

The CT image provides radiodensity in [Hounsefield units](https://en.wikipedia.org/wiki/Hounsfield_scale); lung tissue approaches -1000 in intensity, the value for air, and soft tissue is slightly above 0 intensity, the value for water.

CT影像的放射剂量是[Hounsefield units](https://en.wikipedia.org/wiki/Hounsfield_scale)，肺组织通常在-1000的强度，这个值是属于空气的，软组织是比0大一点，这个值属于水的。

In [3]:
view(fixed_image, vmax=-300, gradient_opacity=0.5, ui_collapsed=True, annotations=False)

Viewer(annotations=False, geometries=[], gradient_opacity=0.5, point_sets=[], rendered_image=<itkImagePython.i…

In [4]:
view(fixed_image, vmin=-50, vmax=500, ui_collapsed=True, annotations=False)

Viewer(annotations=False, geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.…

In [5]:
print(fixed_image.GetSpacing())

view(fixed_image, mode='z')

itkVectorD3 ([1.366, 1.366, 2.5])


Viewer(geometries=[], gradient_opacity=0.22, mode='z', point_sets=[], rendered_image=<itkImagePython.itkImageS…

**Take-home observations**:

- The resolution is anisotropic; resolution is higher in-plane versus out-of-plane.
- The pixel type is signed short: registration requires supporting this pixel type and interpolating samples between these pixels.

- 分辨率是各向异性的；面内分辨率比面外分辨率高。
- 像素类型被签名为short：配准需要支持此像素类型并在这些像素之间插值采样

We **conceptualize** a **spatial image** as a **uniform grid of pixels** or **pixel grid** that lives is physical space.

Let's continue [with this presentation](https://docs.google.com/presentation/d/1Mo_yHmJ7Uai2DKAO2VTyAm56V9KdI-Wx5xVgc9gljmQ/edit?usp=sharing).

### Exercise: Lung image size

There are two 3D CT lung images, sampled on different image grids:

1. *data/CT_3D_lung_fixed.mha*
2. *data/CT_3D_lung_fixed_small.mha*

Which is larger in terms of its:

* File size?
* Spatial domain?
* Resolution?

How much less time will it take to register *CT_3D_lung_fixed_small.mha* to *CT_3D_lung_moving.mha*?

In [9]:
!ls data/CT*mha

data/CT_3D_lung_fixed.mha	 data/CT_3D_lung_moving.mha
data/CT_3D_lung_fixed_small.mha


In [9]:
!head -12 data/CT_3D_lung_fixed.mha | grep DimSize 

DimSize = 115 157 129


### Enjoy ITK!